![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/16.GPT2_Transformer_In_Spark_NLP.ipynb)

# GPT2Transformer: OpenAI Text-To-Text Transformer

![image.png](https://miro.medium.com/max/1400/1*UZTblFMn8oWfmqBMQhUOdA.jpeg)

[GPT-2](https://openai.com/blog/better-language-models/)  displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where the model is primed with an input and it generates a lengthy continuation.

Pretrained models can be loaded with `pretrained()` of the companion object:

**GPU MODE** is highly reccomended, since this is a very large model.

## Colab Setup

In [1]:
! pip install -q pyspark==3.3.0 spark-nlp==4.0.1

     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 531 kB 16.0 MB/s 
     |████████████████████████████████| 199 kB 89.3 MB/s 


In [69]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel



**Make sure to Enable GPU Mode**


In [70]:
# Comment out this line  and uncomment the next one to enable GPU mode
spark = sparknlp.start()

# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.0.1
Apache Spark version: 3.3.0


**GPT2 Models In Spark NLP**

*   `gpt2`
*   `gpt2_medium`
*   `gpt2_distilled`
*   `gpt2_large`

The default model is `"gpt2"`, if no name is provided. For available pretrained models please see the [Spark NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=gpt2)



## GPT2 Pipeline 

Now, let's create a Spark NLP Pipeline with `gpt2_medium` model and check the results.

In [3]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")
    
gpt2 = GPT2Transformer.pretrained("gpt2_medium") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setMinOutputLength(25) \
    .setOutputCol("generation")
    
data = spark.createDataFrame([["My name is Leonardo."]]).toDF("text")

pipeline = Pipeline().setStages([documentAssembler,gpt2]).fit(data)

result = pipeline.transform(data)
result.select("generation.result").show(truncate=False)

gpt2_medium download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ My name is Leonardo. I'm a student at the University of California, Berkeley. I've been studying computer science for the past two years. I have a PhD in computer scienc

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for repetition penalty. 1.0 means no penalty. See `this paper <https://arx

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [ ]:
sample_texts= [[1, "Mey name is  Leonardo"], 
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"], 
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [ ]:
gpt2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                           |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it True that Mey name is Leonardo da Vinci ?\n\nAnswer: Leonardo da Vincenzo was not a member of the Order of the Phoenix and it is true that Mezco da Vinco was not part of that group.]                   |
|2  |[ Is it True that My name is Leonardo and I come from Rome.\n\nLeonardo: My name isn't Leonardo.\n. . .\n\nSister: I have come to g

### Changing the Transformer's task

Now, we change the task of Transformer. We can verify some informations to GPT-2 by setting `.setTask()` parameter as **"Is it true that"**. <br/>
We give a text to the model by setting `setTask("Is it true that")` and model adds the "Is it true that" expression at the beginning of the sentence and generates sentences.

In [ ]:
gpt2.setTask("Is it true that").setMaxOutputLength(50)

In [ ]:
sample_text= [[1, "Donald Trump is rich?"],
              [2, "Pink Floyd is rock band?"]]

sample_df= spark.createDataFrame(sample_text).toDF("id", "text")

result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                          |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it true that Donald Trump is rich? He's not the richest person in the world—he certainly isn't the wealthiest person in America, either. He's got $1.2 billion in assets. But he's not a millionaire, so to]                               |
|2  |[ Is it

# Some Helper Methods for generating and printing generation results

In [5]:
from typing import List
def prompts_to_spark_df(prompts,spark=spark): 
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")  

def generate_with_gpt2(gpt_pipe, prompts,task='generate', print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else : 
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = pipeline.transform(df)
  df = df.select("id",'text', "generation.result").toPandas()
  
  if print:
    print_generation_results(df)
  return df 

def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore GPT Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words. 
The probability distribution is conditioned on all previous tokens in a text to generate the next token. 

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions : 

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters : 

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable. 


These parameters are shared by all method : 
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once 
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf> 
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

In [40]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 100)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for repetition penalty. 1.0 means no penalty. See `this paper <https://ar

### Play with temperature 
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `gpt2.setDoSample(True)` to have non-deterministic results

In [36]:
text = """Hello my name is GPT2, I love to """
data = [text, text,text,text,text ]
gpt2.setMaxOutputLength(200)


GPT2TRANSFORMER_729a8e69e804

In [43]:
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to  do this work very much , my job is hard but rewarding and I always am happy to try whatever I am going for . I am a passionate collector of old school cars from all around the world   and from all corners of the globe. All items come with a complete list of information on them, so you will get a full visual and an up-to-date list of every car shown at the event in this article.
I bought this car for my friend Chris and he had told me it would be difficult for him to afford to buy a car of similar price range. I was excited to see this car out there in the sunlight and I loved the idea that I could own a car like that and be ready to go when everything was done. So far my friend has had almost everything on the road – all the wheels, suspension

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to do this wo..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to irl dress s..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to _______ and..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to _______ and..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to work hard ..."


In [38]:
gpt2.setTemperature(0.75)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to ive done this and still do. I am a big fan of the 3D printing industry. I have done the 4D print project using a few 3D printers and I am an enthusiast. I'm currently using 3D print kits from hobbyists. I've been working on my own plans for the 3d print project. All of the pictures are of models I've made in my own home. I just finished my 4D printer and I've got my own setup. I can't wait to see what your plans are for me, I'm so excited! Thanks for taking the time to visit my project!


Hi, my name and email is Gpt2 . I love 3D printed stuff and I want to show you my experience. I know its a lot easier to print on 3D printer when you have a lot of patience, and a lot less effort. I hope this helps!

 "I wish I could


Example 1: _____________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ive done th..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to 【Hakurei Sh..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to _____.\n\nW..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ____ and I ..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play MMORP..."


In [39]:
gpt2.setTemperature(0.5)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to xtreme! I have been playing for a long time now and am looking for some new challenges. My main goal is to move up in ranks, and I want to do that by doing more and more of the same. I want more challenges, and more challenges! I want my gear to be better and better and I'm looking for people who will help me in that effort.


I am a very active player, and will be doing a lot of the following:

- Playing with my friends on the PVE server, and trying out new things

(I will also be doing some PVP, but I'm not sure if I can handle it without a dedicated server)

I will be taking a lot more time to post updates, so please be patient. I am really sorry if you are waiting for me to reply to your PM, but please do not be upset if I don't respond to 

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to xtreme! I h..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to _________ a..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play and ..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to irl and irl..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to irl and I l..."


In [40]:
gpt2.setTemperature(0.25)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline,data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to  play games. I am a gamer, I am an avid gamer, and I am also a person who loves to play games.
I am a person that loves to  at least try new games.  I am also an avid player of  games, and this is my passion. :)
I have a passion for  the  game  and I am always looking for new games to play.  :))
I love to play  my  favorite     games.    :))  My  favourite  Games are  The Legend of Zelda,  Super Mario Bros, ʼs  Mario Kart,   FIFA, ˆ ˇ ˈ ˌ ˉ ˊ ˋ ˍ ˎ ˏ ː ˝


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to  play games and I am very active on the forums.

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play games..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play games..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to read and w..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play and I..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ive been pl..."


In [41]:
gpt2.setTemperature(0.1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to  play FFXIV. I never realized yet how serious this might be. Maybe GPT3 if ever available."
Bugsbear's "GPT2" PVP Guide Part 20 By: GPT1
New Season Two bugsbear's P4P Guide A guide to Part 20 of the new season 2 bugsBugsBear's PPA Guide Patch Update 23 Part:18 patch The patch introduces a multitude change making most monsters for the Mists of Pandamon-Lok. First of all, these newly nerfed T1 and T2 monsters which introduced such a harsh "Giant" system during the 4th patch could be brought higher this patch before there is this difficult PVP (PVP against Player 4) at all. T1 monsters at the main is on the front now meaning higher PvE risk factor: less farming + more high risk gameplay without risking pvp of late death rate being 100%. Players




,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play FFXIV..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!!\n\nHAND..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!!\n\nHAND..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to !!!\n\nHAND..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to play FFXIV..."


In [35]:
gpt2.setTemperature(0.0001)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to ҹand͘ talk i did make someone pay [HypeCupid's image]-♹◴Hahaha…❁i️love all c*** of his sex ❔murdro!! and there r very only of girls we'd feel truly fucked without th❹thing happening —galt lernage hos! a tis better? when in its first 30 moments every question looks similar from here –


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Hello my name is GPT2, I love to ҹand͘ talk i did make someone pay [HypeCupid's image]-♹◴Hahaha…❁i️love all c*** of his sex ❔murdro!! and there r very only of girls we'd feel truly fucked without th❹thing happening —galt lernage hos! a tis better? when i

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
1,1,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
2,2,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
3,3,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."
4,4,"Hello my name is GPT2, I love to","[ Hello my name is GPT2, I love to ҹand͘ talk ..."


# GPT2 Use Cases

These are some examples to give you an idea of what GPT2 is capable of 

## GPT for Programing Code

### GPT and Scala

In [64]:
# Hello world in Scala
scala_code = """
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, scala_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
Here, we call main with an array, which we pass to the println method. The "Hello, World" will appear in the output list. We are passing an Array[Object] as an input argument. The next thing to note is that we are declaring an instance of java.lang.Object , which is similar to using an object literal. This is important for a couple reasons:
- If you are importing a class from another source, you cannot use method names from other sources without changing the name of the instance of your class.
You can also use a reference by using a method, rather than referring to that method directly with the name object. In this case, we are calling println . Once done with that statement, the outp

,id,text,result
0,0,\nobject Hello {\n def main(args: Array[Str...,[ \nobject Hello {\n def main(args: Array[S...


### GPT and Python

In [66]:
# adding 3 numbers in python
py_code = """
def my_func(a,b,c):
  return a+b+c
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, py_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
def my_func(a,b,c):
  return a+b+c

class Example:

@classmethod

def hello_world(obj):

return obj[1]

# <class:obj1, instance:my_type>

a = Example.hello_world()

b = Example

c = my_type (a, b)

print(c.add_one())

__main__

(foo,bar)
 (123)



--

Output

-- <class, instance>: <class.foo>
.foo: Bar, Example
 # >

Works with: Python 2.7

1 2 3 4 5 1 2 3 . 6

--- example/hello_hello.py < input > , : , < main > , hello_hello

from r2.models import Example
 "example.hello"

namespace Example
 [ example / hello_Hello ]

add_method :

_ add ( example , example . add_one , example

. add_two , example )

instance = example . instance

new_method = example = example ():
 "hello the world"
 Heap :
 ( 1 )
. set ( 0 ) Heap . new {

append ( add_method , example : add_some_method )



,id,text,result
0,0,"\ndef my_func(a,b,c):\n return a+b+c\n","[ \ndef my_func(a,b,c):\n return a+b+c\n\ncla..."


In [67]:
# breadth-first search 
bfs_python_code = """
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0) 
    print (s, end = " ") 

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
"""


gpt2.setMaxOutputLength(1000)
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, bfs_python_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0) 
    print (s, end = " ") 

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
    
When iterating through the whole queue, when iterating an odd numbered queue, the first returned node is the neighbour on which we iterate.
A function called node() takes us through the first element of the queue as argument:
print "Listing list of neighbors in queue."
For each element in the queue, call the function "next" which returns the neighbour with the value "next".
This function is called again and again and only returns the first result of its iterators.
For the sake of this explanation, let us

,id,text,result
0,0,"\ndef bfs(visited, graph, node):\n visited.ap...","[ \ndef bfs(visited, graph, node):\n visited...."


## Generate Product Reccomendations

We can start a base list of top 10 anything and GPT2 will complete it for us!      
This is very useful to reccomendation and search engines.

### Top 10 Movie List

In [51]:
movie_list = """My top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4."""

gpt2.setTemperature(1)
gpt2.setMaxOutputLength(200)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.  The Lord of the Rings: The Fellowship of the Ring 1
5. Inception 1
6. Independence Day: Resurgence 1
7. Transformers 1
8.  Man of Steel 1
9. Avatar: The Last Airbender 1
10.  Star Wars Episode VI: Return of the Jedi




,id,text,result
0,0,My top 10 movie list : \n1. The Matrix 1\n2. T...,[ My top 10 movie list : \n1. The Matrix 1\n2....


How **not** to generate text with GPT2

Since every new word is conditioned on every previous **character**, things like **white spaces** and **new lines** can skew the sampling distribution into an unintended direction

In [48]:
# NOTE! We have malformed the input text and added a newline. 
# This will Confuse GPT2, it might think the list is over and probability distribution changes accordingly
movie_list = """My Top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. 

"""
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. 


We had another epic movie night which was well received by both critics and TV viewers. It was amazing to watch the new trailers for both Terminator: Salvation and Terminator: Genesis and hear all kinds of rumors online.

Finally, we had another good night with some great reviews. Some of my favorites were The Walking Dead and Game of Thrones. I'm just going to focus on the series as it has a special place in my heart! It's my favorite show on TV right now, and I'll go right out on a limb here and say that The Walking-Dead is my absolute favorite show of all time. It's got The Walking in it, the death of Daryl, his family in an insane and completely peaceful suicide, and the group bonding from Rick-O

,id,text,result
0,0,My Top 10 movie list : \n1. The Matrix 1\n2. T...,[ My Top 10 movie list : \n1. The Matrix 1\n2....


In [52]:
movie_list = """My Top 10 worst movie list : 
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 worst movie list : 
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4.  The Shining 3
5.   Mockingjay Part 2
6.     Honey and the Devil
7.                             
8.                Star Wars II
9.  The Terminator
10. 
I'm not going to go through and mention every movie off the top of my head, but I'll list the 10 worst films off of the top. 10.  Snow Crash (2000) 10.   Snow Crash – a parody of the first Snow Crash and a sequel – is really an anti-movie which I am totally on the fence about. I have seen Snow Crash 4x before and I have no problem seeing Snow Crash 5x. I understand Snow Crash's flaws and have been to Snow Crash parties in my life




,id,text,result
0,0,My Top 10 worst movie list : \n1. The Matrix 4...,[ My Top 10 worst movie list : \n1. The Matrix...


### Top 10 Game List

In [59]:
game_list = """My Top 10 video games of all time list : 
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 video games of all time list : 
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4.  Witcher 3  (by the way, I'm not kidding when I say I own several dozen copies of the Witcher 3, which means I've played it about as many as everyone who knows any words. My husband's only played a few hours in this game.)
5. Half Life 2
6. Counterstrike: GO
7. Quake 3 (and Counter Strike 2)
8. Dota 2
9.  Grand Theft Auto V
10.   Final Fantasy IV
I've already made room for Portal 4 in my top 10. This will be my first video ever. What are my favorite RPG's? I love a good RPG and the likes of the Elder Scrolls series and Shadowrun and Planescape.  We still have about 40 or so classics, but you can definitely pick an RPG




,id,text,result
0,0,My Top 10 video games of all time list : \n1. ...,[ My Top 10 video games of all time list : \n1...


### Top 10 Book List

In [62]:
game_list = """My Top 10 books of all time list : 
1. 1964
2. Moby Dick
3. robinson crusoe
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 My Top 10 books of all time list : 
1. 1964
2. Moby Dick
3. robinson crusoe
4.  The Man Who Knew Too Much
5. The Hitchhiker's Guide to the Galaxy
6. the bible
7.  A Tree Grows in Brooklyn
8.  The Lion King
9. The Great Gatsby
10.  A View of the Horse
Why these books are  Top 10 book of all times 
A. They are great books all on their own! Each is a triumph in its own category, including:
1) Moby-Dick , this novel was one of the biggest breakthroughs for a science fiction writer. It is about Moby and Dick's trip to the moon , and is a brilliant look at how human imagination, a passion for adventure and an appreciation for nature have all led them to our lunar outpost. 
2) The Hitch-hiker Guide




,id,text,result
0,0,My Top 10 books of all time list : \n1. 1964\n...,[ My Top 10 books of all time list : \n1. 1964...


## SEO Text / Content Writing / Copy Writing
Lets generate some texts for our new imaginary shopify store!
We will sell some hoodies, soap and beard products.

Based on some base templates, we can generate high quality marketing/seo texts

In [55]:
# from https://www.shopify.com/blog/8211159-9-simple-ways-to-write-product-descriptions-that-sell 

# Lets define some base texts, from which we will generate new texts

hoodie = """Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! """

soap  = """Made with real pine extract, this all-star bar is as tough as a freshly cut bat. 
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent."""

beard = """ Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, """
gpt2.setTemperature(1)
gpt2.setDoSample(True)



GPT2TRANSFORMER_729a8e69e804

#### Marketing Texts for Hoodies

In [60]:
# Lets generate some marketing texts for our hoodies
generate_with_gpt2(pipeline, [hoodie,hoodie,hoodie,hoodie,hoodie,hoodie] ,task='generation', print=True)


Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Cant stop buying plants? Unbeleafable. Dont worryus too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie!  I bought this bag to hang it on my cupboards to give it a more playful feel. it keeps my hands warm when I sit in the cold but not too warm!   
this is a very interesting pattern and it's super fun to play with because you have to move around and make different pieces to create your own pieces to see which you like the best.   For example this is my own copy of a baby daisy pattern and although it looks much better as a patterned piece this isn't an actual daisy.  I really don't care for that because I find daisies a bit distracting! 
I love this little trick because it does two things!   It keeps you from trying to figure out which item you like

,id,text,result
0,0,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Cant stop buying plants? Unbeleafable. Dont ...
1,1,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Cant stop buying plants? Unbeleafable. Dont ...
2,2,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Cant stop buying plants? Unbeleafable. Dont ...
3,3,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Cant stop buying plants? Unbeleafable. Dont ...
4,4,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Cant stop buying plants? Unbeleafable. Dont ...
5,5,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Cant stop buying plants? Unbeleafable. Dont ...


#### Marketing Texts for Soap

In [61]:
# Lets generate some marketing texts for our soap
generate_with_gpt2(pipeline, [soap,soap,soap,soap,soap,soap] ,task='generation', print=True)


Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Made with real pine extract, this all-star bar is as tough as a freshly cut bat. 
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent. When this product is on a humid or oily day it'll have a deep, velvety scent that is easy to wear in the shower.  TESTIMONIALS -  THE NESTLE-WHISKED PINE BEACH
DRAFT & TRIPPED - I knew a good pine-whisked shower bar was on the rise (and I was right). In the last few months I've decided that the best time to use and share my tips on how to use pine bar is when my partner takes my hand and tells me how to start it.   The Pine-Whisked Bathtub in this scent is like a deep dish filled with warm-tasting pine, soft and


Example 1: ______________________________________________________

,id,text,result
0,0,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
1,1,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
2,2,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
3,3,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
4,4,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."
5,5,"Made with real pine extract, this all-star bar...","[ Made with real pine extract, this all-star b..."


#### Marketing Texts for beard products

In [71]:
# Lets generate some marketing texts for our beard products
generate_with_gpt2(pipeline, [beard,beard,beard,beard,beard,beard] ,task='generation', print=True)


Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control,  with little hold for hair on the sides and mid-part for good control. The shape also provides a good grip for hair that grows on the crown and sides, and it stays in place for hours, not just on your cheeks and chin.  If you already have thin, brittle hair, it won't really do much, if it at all.  However, if thick, wavy hair is added to a tangle, then Beardbrand's Curled and Wavy will work wonders.
For a beard, it works wonders on the thick, thin hair, where the curl would go away.  It is great for curly-end hair, and its shape also adds some nice contouring and finishing around the chin if des

,id,text,result
0,0,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
1,1,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
2,2,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
3,3,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
4,4,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."
5,5,"Whatever your style is, Beardbrand Styling Ba...","[ Whatever your style is, Beardbrand Styling B..."


## Song Lyrics

#### Last Christmas

In [68]:
last_christmas = """
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, last_christmas,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
A man - you know why
- who'll appreciate it and will tell my mom
When you hear me cry, you'll go
Cause I know these words aren't on my lips
And they're not on your tongue
But I know what it feels like to hold a thing sacred
And so I'm here on Santa to let you go
I hope every single day is just like Christmas
Just like Santa, but without the tears
Every day is like Christmas, just like Santa
- yeah, even the snowman
But it's Christmas, you know I do
I'm just gonna miss you every night
Until you're gone
Just give me a gift and I believe you could do it together




,id,text,result
0,0,"\nLast Christmas, I gave you my heart\nBut the...","[ \nLast Christmas, I gave you my heart\nBut t..."


#### Fresh prince of GPT

In [25]:
bel_air = """
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, bel_air,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 generate 
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
And how I got my first kiss
And then I got a second
And a third
And now I'm the king of a kingdom
And my kingdom is called
Bel-Air.
I'm the prince, and I'm a king
I've got a lot of power
And if I don't get my hands on
My kingdom's gold
I might as well just die
I don't want to be a king anymore
I want to live in a castle
And be a prince
And have a princess
And marry a princess and have a baby
And live in the castle
I just want to get my gold
And get my kingdom's crown
And make my kingdom great again
I can't wait to see what happens
I know I'm going to be the king
And the princess
I l

,id,text,result
0,0,"\nNow, this is a story all about how\nMy life ...","[ generate \nNow, this is a story all about ho..."


#### GPT Jackson

In [35]:
thriller_lyrics = """
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
"""


gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, thriller_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 generate 
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
Don' wanna see blood, you gotta be strong
You gotta be tough and you gotta do what's right
You can't be weak, you can't hide
You got to be strong, you got to do what it takes
You know you got a chance, you know you gotta fight
You'

,id,text,result
0,0,"\nThey told him, ""Don't you ever come around h...","[ generate \nThey told him, ""Don't you ever co..."


#### Real Slim GPT

In [36]:
real_slim_shady = """
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
"""



gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, real_slim_shady,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 generate 
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
I'm gonna have to get a new one, I'm gonna get a different one
I don't know what to do, I don't even know what I'm doing
I can't believe I'm still here, I can't ev

,id,text,result
0,0,"\nMay I have your attention, please?\nMay I ha...","[ generate \nMay I have your attention, please..."


#### GPT Rap God Bot

In [37]:
rap_god_lyrics = """
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot """



gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, rap_god_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 generate 
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot


I don't care if you think I'm a robot

I ain't no robot, I ain't a robot


You think I rap, you think you rap

Yo

,id,text,result
0,0,"\nLook, I was gonna go easy on you not to hurt...","[ generate \nLook, I was gonna go easy on you ..."


## Fiction Writing

#### Fantasy Stories

In [28]:
unicorn_story = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, 
previously unexplored valley, in the Andes Mountains. 
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""



gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, unicorn_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 generate In a shocking finding, scientist discovered a herd of unicorns living in a remote, 
previously unexplored valley, in the Andes Mountains. 
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.  The researchers were able to translate the words of the unicorn, which were written in the language of the animals.  
The researchers also discovered that the animals were able, in fact, communicate with each other.  The researchers were also able to identify the animals' names, which are written in a language that the researchers believe is a combination of English and Spanish. 




,id,text,result
0,0,"In a shocking finding, scientist discovered a ...","[ generate In a shocking finding, scientist di..."


In [31]:

silly_story = """
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and 
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!"""


gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, silly_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 generate 
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and 
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!
The Spanish language is a complex language with many different forms, but the most common form is the 
Spanish alphabet. The Spanish alphabet is a combination of letters from the Latin alphabet and letters from other languages.
The letters in the Spanish alphabet are arranged in a row, with the first letter being the letter that is the first character of

,id,text,result
0,0,\nBlue creatuers have been found in undergroun...,[ generate \nBlue creatuers have been found in...


#### Lovecraftian Horror Stories 

In [72]:
lovecraft_text = """
At this horror I sank nearly to the lichened earth, 
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame, 
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected, 
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants, 
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, lovecraft_text,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
At this horror I sank nearly to the lichened earth, 
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame, 
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected, 
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants, 
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall. 

They were the strange bird-chases now called crows.  
Distant upon the horizon to the west, they moved, like little birds of prey, out 

,id,text,result
0,0,\nAt this horror I sank nearly to the lichened...,[ \nAt this horror I sank nearly to the lichen...


In [75]:
cthulu_intro = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. 
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. 
The sciences, each straining in its own direction, have hitherto harmed us little"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, cthulu_intro,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. 
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. 
The sciences, each straining in its own direction, have hitherto harmed us little; and we know there is no man among us who would dare to dream of setting foot upon them.  In this present world, however, we are confronted with the difficulty of having anything of a rational origin, so that, as it were, to understand our place in an ever increasing universe of knowledge is an experience to be cherished; a sentiment that we ought not altogether fail to entertain. ‎ Appears in 54 books from 1858-2008

Page 25 I felt an irresistible impulse 

,id,text,result
0,0,"\nThe most merciful thing in the world, I thin...","[ \nThe most merciful thing in the world, I th..."


#### Re-Write movie scripts

In [76]:
the_matrix_script = """
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  The Architect made you. Once he was gone, you lived until you were forty, then you died, but you are here.  You have the Matrix.  

Who, Me?
 , or something like it.  Or you may be thinking of another mind, someone in your memories, someone who could, if you looked hard enough, find where he is currently, s

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\n\nNeo: Who are y...","[ \nThe Architect: Hello, Neo.\n\nNeo: Who are..."


In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 generate 
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: 


The Matrix is a computer simulation of the real world. It is a simulation of a world that is not real. 

 The Matrix has been created by the Architect, and it is the Architect's creation.  The Architect has created the Matrix, and the Matrix has created you.
The matrix is a virtual reality, a virt

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\nNeo: Who are you...","[ generate \nThe Architect: Hello, Neo.\nNeo: ..."


GPT2TRANSFORMER_729a8e69e804